In [4]:
def closure(R, FD, S):
    '''
    Check each functional dependency X -> Y on each iteration and add Y to the closure iff
    X is a subset of the closure formed so far, repeat until there is no change in the set
    '''
    if not (is_subset(S, R)):
        return []
    result = list(S)
    changed = True
    while (changed):
        changed = False
        for fd in FD:
            LHS = fd[0]
            RHS = fd[1]
            if (is_subset(LHS, result)):
                for att in RHS:
                    if (att not in result):
                        result.append(att)
                        changed = True
    return sorted(result)


def is_subset(X, Y):
    '''
    Check if the list is subset of another list
    '''
    return set(X).issubset(set(Y))


def candidate_keys(R, FD):
    '''
    Calculate closure of each attribute susbset of the schema unless the subset is superkey
    but not candidate key. Check in each iteration if the attribute closure equals to R,
    that is, check for candidate key, and if it is not minimal then just continue
    with the next iteration instead of appending closure to the result set
    '''
    result = []
    ck = []
    for att_set in subsets(R):
        att_closure = closure(R, FD, att_set)
        if (att_closure == R):
            is_super = False
            for key in ck:
                if is_subset(key, att_set):
                    is_super = True
            if is_super:
                continue
            ck.append(att_set)
    ls_ck = []
    for i in ck:
        s = ""
        for j in i:
            s += j
        ls_ck.append(s)
    return ls_ck


def subsets(R):
    '''
    Calculate all possible subsets of given list
    Return them in a sorted way (first alphabetic and then length)
    '''
    x = len(R)
    masks = [1 << i for i in range(x)]
    result = []
    for i in range(1, 1 << x):
        r = []
        for mask, ss in zip(masks, R):
            if i & mask:
                r.append(ss)
        result.append(r)
    result.sort()
    result.sort(key=len)
    return result


def prime(ck):
    '''
        takes set of attributes of candidate key and returns set of prime attributes
    '''
    pr = set()
    for i in ck:
        for j in i:
            pr.add(j)
    return pr


def non_prime(pr, R):
    '''
        takes prime attributes and R and returns set of non-prime attributes
    '''
    return R.difference(pr)


r=input('Enter the attributes:')
# r = "a,b,c,d,e,f,g,h"
relation = r.split(",")

l = set()
m = set()
r = set()
fds = []
f1 = []
n=input("Enter functional Dependency:")
# n = "ch->g,a->bc,b->cfh,e->a,f->eg"
li = n.split(",")

for i in li:
    li1 = i.split("->")
    fds.append(li1)
    f1.append(li1)

for fd in fds:
    for i in range(2):
        ls = []
        for j in fd[i]:
            ls.append(j)
        fd[i] = ls

ck = candidate_keys(relation, fds)
print("-------------------------------------------------------------------")
print("----------------------------CANDIDATE KEYS--------------------------")
print("Candidate keys are :")
for i in ck:
    print(i)
print("--------------------------------------------------------------------")
print("---------------------------PRIME ATTRIBUTES-------------------------")
pr = prime(ck)
for i in pr:
    print(i)
print("--------------------------------------------------------------------")
print("------------------------NON-PRIME ATTRIBUTES-------------------------")
np = non_prime(pr, set(relation))
for i in np:
    print(i)
print("---------------------------------------------------------------------")


def check(string, sub_str):
    if (string.find(sub_str) == -1):
        return 0
    else:
        return 1


def check_bcnf(ck, fd):
    ls_left = []
    ls_right = []
    flag = 0
    for fd in fds:
        sl = ""
        for i in fd[0]:
            sl += i
        ls_left.append(sl)

        sr = ""
        for i in fd[1]:
            sr += i
        ls_right.append(sr)

    for i in ls_left:
        for j in ck:
            if (check(i, j) == 0):
                print("Violating BCNF Condition at {}->{}".format(i, ls_right[ls_left.index(i)]))
                flag = 1
                break
    print("--------------------------------------------------------------------------")
    if flag == 1:
        return False
    else:
        return True


def in_np(s, np):
    for i in s:
        if i in np:
            return 0
    return 1


def check_3nf(ck, fd, R):
    ls_left = []
    ls_right = []
    flag = 0

    pr = prime(ck)
    np = non_prime(pr, set(R))
    #     print(np)
    #     print(pr)

    for fd in fds:
        sl = ""
        for i in fd[0]:
            sl += i
        ls_left.append(sl)

        sr = ""
        for i in fd[1]:
            sr += i
        ls_right.append(sr)

    for i in ls_left:
        for j in ck:
            if check(i, j) == 0 and in_np(ls_right[ls_left.index(i)], np) == 0:
                print("Violating 3NF Condition at {}->{}".format(i, ls_right[ls_left.index(i)]))
                flag = 1
                break
    print("--------------------------------------------------------------------------")
    if flag == 1:
        return False
    else:
        return True


def check_2nf(ck, fd, R):
    ls_left = []
    ls_right = []
    flag = 0

    pr = prime(ck)
    np = non_prime(pr, set(R))
    for fd in fds:
        sl = ""
        for i in fd[0]:
            sl += i
        ls_left.append(sl)

        sr = ""
        for i in fd[1]:
            sr += i
        ls_right.append(sr)

    for i in ls_left:
        for j in i:
            if j in pr and ls_right[ls_left.index(i)] in np:
                print("Violating 2NF Condition at {}->{}".format(i, ls_right[ls_left.index(i)]))
                flag = 1
    print("--------------------------------------------------------------------------")
    if flag == 1:
        return False
    else:
        return True


print("--------------------------------------------------------------------")
if check_bcnf(ck, fds) == True and len(ck)!=0:
    print("Relation is in BCNF")
elif check_3nf(ck, fds, relation) == True and len(ck)!=0:
    print("Relation is in 3NF")
elif check_2nf(ck, fds, relation) == True and len(ck)!=0:
    print("Relation is in 2NF")
else:
    print("Relation is in 1NF")


Enter the attributes:A
Enter functional Dependency:A
-------------------------------------------------------------------
----------------------------CANDIDATE KEYS--------------------------
Candidate keys are :
ad
bd
de
df
--------------------------------------------------------------------
---------------------------PRIME ATTRIBUTES-------------------------
e
b
f
d
a
--------------------------------------------------------------------
------------------------NON-PRIME ATTRIBUTES-------------------------
g
c
h
---------------------------------------------------------------------
--------------------------------------------------------------------
Violating BCNF Condition at ch->g
Violating BCNF Condition at a->bc
Violating BCNF Condition at b->cfh
Violating BCNF Condition at e->a
Violating BCNF Condition at f->eg
--------------------------------------------------------------------------
Violating 3NF Condition at ch->g
Violating 3NF Condition at a->bc
Violating 3NF Condition at b->cfh
